In [1]:
from twitch_listener import utils
from twitch_listener import listener

In [2]:
# Connect to Twitch
# bot = listener.connect_twitch('twitch_pioneers', 
#                              'oauth:lolbsmiac5expvax1iqiysfo18hqi2', 
#                              '6resxnu2ehi2ggn0kgpue6g12bxtlw',
#                              'zkekfhyhzhdl8ltxs6b8jdez7b2i6b')

nickname = 'twitch_pioneers'
oauth_chat = 'oauth:lolbsmiac5expvax1iqiysfo18hqi2'
client_id = '6resxnu2ehi2ggn0kgpue6g12bxtlw'
oauth_api = 'Bearer y5o8h4s2fljc1s632ylfrb540npcs0'


bot = listener.connect_twitch(nickname, 
                             oauth_chat, 
                             client_id,
                             oauth_api)

chat_file_path = '/home/w210/Twitch-chat-pioneers/chat_listener'

# List of channels to connect to
# Channel names must be lowercase
channels_to_listen_to = ['xqc','sinatraa']

In [3]:
#returns list of live channels
channels_to_listen_to = utils.is_live(channels_to_listen_to)

channels_to_listen_to = utils.get_broadcast_id(channels_to_listen_to, client_id, oauth_api)
print(channels_to_listen_to)

{'xqc': '71092938', 'sinatraa': '138094916'}


In [4]:
# Scrape live chat data into raw log files. (Duration is seconds)
bot.listen(channels_to_listen_to, duration = 25, until_offline = True, debug = False) 

# Convert log files into .CSV format
#bot.parse_logs(timestamp = True, file_path = chat_file_path)

# Generate adjacency matrix
#bot.adj_matrix(weighted = False, matrix_name = "streamer_network.csv")

KeyboardInterrupt: 

In [5]:
import sqlite3
conn = sqlite3.connect('db.sqlite3')

In [6]:
#test = conn.execute('''drop table if exists chats_table''')
#test = conn.execute('''select * from chats_table order by date desc limit 100''')
test = conn.execute('''select count(*) from chats_table''')
test.fetchall()

[(1205,)]